In [1]:
!pip install vk

You are using pip version 18.1, however version 21.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### Как начать использовать VK API?

**1. Создаем приложение**

Нужно перейти по адресу https://vk.com/apps?act=manage и создать *Standalone* приложение (потребуется подтвердить действие в телефоне), а затем в настройках узнать *сервисный токен* (редактировать -> настройки). Он понадобится для отправки запросов.

**2. Изучаем документацию**

https://vk.com/dev/manuals и, в частности, https://vk.com/dev/first_guide.

По каждому методу в разделе "список методов" снизу есть список параметров и пример возвращаемых данных.

**3. Посылаем запросы**

Отправляем запросы по адресу https://api.vk.com/method/ИМЯ-МЕТОДА/, передавая все нужные параметры, включая *accesstoken* (ваш сервисный токен) и *v* - версию API (можно подсмотреть в примерах).

#### Авторизация в VK API

Для авторизации в API VK вам нужно получить **токен** - ваш временный пароль, по которому vk поймет, что вы - это вы.

Токены в вк бывают трех видов:
* **Сервисный** (можно узнать в настройках [приложения](https://vk.com/apps?act=manage)) - самый простой токен, позволяет получать только открытую информацию;
* **Пользовательский** (можно получить через oAuth) - позволяет делать произвольные действия от **вашего** имени;
* **Сообщества** (можно получить в настройках вашего паблика) - позволяет делать некоторые действия от имени паблика, например, отвечать на сообщения.

**Как получить сервисный токен?**

Нужно создать [standalone приложение](https://vk.com/apps?act=manage) и узнать его в настройках (кнопка редактировать) вашего приложения.

**Как получить пользовательский токен?**

Чтобы получить пользовательский токен, достаточно перейти по ссылке, разрешить доступ и скопировать токен из адресной строки страницы, на которую вы попадете:

https://oauth.vk.com/authorize?client_id=7464780&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends,photos,audio,video,docs,notes,pages,status,wall,groups,notifications,offline&response_type=token

https://oauth.vk.com/blank.html#access_token=ТОКЕН_ТУТ&expires_in=0&user_id=7980360


Вместо `7464780` можно указать `id` вашего приложения, а в `scope` указать только нужные параметры.

**Как получить токен сообщества?**

Его можно разделе *Работа с API* управления сообществом.

### Получение данных

Давайте попробуем использовать метод [wall.get](https://vk.com/dev/wall.get). Чтобы его вызвать, нам понадобится:
* **domain** (имя страницы из адресной строки) или **owner_id** (id страницы);
* **count** - число записей, которые мы хотим получить;
* **access_token** - наш сервисный ключ доступа;
* **v** - версия api.

Если мы хотим взять 10 записей из [паблика кинопоиска](https://vk.com/kinopoisk), получится вот такой адрес:

`https://api.vk.com/method/wall.get?domain=kinopoisk&count=10&access_token=ТУТ_ВАШ_ТОКЕН&v=5.74`

За один раз можно получить не более 100 записей, но можно получать следующие (и следующие 100), делая новые запросы и передавая параметр `offset` (0, 100, 200 и до тех пор, пока не скачаны все записи).
 
Если набрать его в браузере, мы увидим JSON ответа:

![](https://storage.geekclass.ru/images/c2b2b424-843a-47aa-8986-416bb006ad9d.png)

Используем `requests` и `json`, чтобы вывести это в удобном формате:

```python
import requests, json


domain = 'kinopoisk'
count = 10
token = 'ТУТ ВАШ ТОКЕН'

url = "https://api.vk.com/method/wall.get?domain={}&count={}&access_token={}&v=5.74".format(domain, count, token)

response = requests.get(url)
answer = json.loads(response.text)

for post in answer['response']['items']:
    print(post['text'] + '\n')
```

Аналогичную операцию можно сделать с использованием библиотеки `vk`.

In [4]:
import vk

TOKEN = ""

session = vk.Session(access_token=TOKEN)
api = vk.API(session)

answer = api.wall.get(domain='kinopoisk', v=5.21)

print(answer)

{'count': 56830, 'items': [{'id': 2766073, 'from_id': -108468, 'owner_id': -108468, 'date': 1619408493, 'marked_as_ads': 0, 'post_type': 'post', 'text': '«Оскар» — все, и наша трансляция тоже. Но с сайта она никуда не денется, заходите перечитывать самые интересные моменты церемонии:', 'is_pinned': 1, 'attachments': [{'type': 'link', 'link': {'url': 'http://www.kinopoi.sk/69Zp', 'title': '«Оскар-2021»: Прямая текстовая трансляция с церемонии — Статьи на КиноПоиске', 'description': '', 'target': 'internal', 'image_src': 'https://sun9-65.userapi.com/impg/c853624/v853624117/25874f/ZPFBfb-xq3o.jpg?size=150x80&quality=96&crop=87,0,900,480&sign=2f69e76b4b53919be64cf5668e18e40c&c_uniq_tag=tw1grlDukYKweExGmLnq3_muHbQ_WVF2t4nb_wnnvmc&type=share', 'image_big': 'https://sun9-65.userapi.com/impg/c853624/v853624117/25874f/ZPFBfb-xq3o.jpg?size=537x240&quality=96&sign=0be9b1601a6eee463954a2f3f9343d9f&c_uniq_tag=8fJ8dPQT58nncelV4YHgPrnI6WarrSlDP_iOuWiX79A&type=share'}}], 'post_source': {'type': 'api'}